# CHEM 1000 - Spring 2022
Prof. Geoffrey Hutchison, University of Pittsburgh

## Recitation Ch 8 - Numeric Integration

**Learning Objectives**

- Understand how to perform numeric integration (using a few different techniques)
  - Understand why some methods work better than others

**Attribution**

Much of this material has been adapted from [Mathematical Python by Patrick Wills](https://github.com/patrickwalls/mathematical-python/blob/master/integration/riemann-sums.ipynb)

### Numeric Integration

Sometimes we need to use numeric integration rather than analytical indefinite or definite integrals:
- The function / data we want to integrate doesn't have a formula (e.g., NMR spectra, MRI images, real-world data, etc.)
- The function we want to integrate has no known analytical solution
    - (e.g., electron density on a grid, machine learning models
- The function has an analytical form, but "it's complicate" and maybe it's faster or better to do the integration numerically

Remember that we initially expressed integrals as the area under a curve - a limit of a sum of rectangles.

These are known as [Riemann sums](https://en.wikipedia.org/wiki/Riemann_sum), which important because they provide an easy way to approximate a definite integral

$$
\int_a^b f(x) \, dx \approx \sum_{i=1}^N f(x_i^ * ) (x_i - x_{i-1}) \
$$

Notice that the product $f(x_i^ * ) (x_i - x_{i-1})$ for each $i$ is the area of a rectangle of height $f(x_i^ * )$ and width $x_i - x_{i-1}$. We can think of a Riemann sum as the area of $N$ rectangles with heights determined by the graph of $y=f(x)$.

The value $x_i^*$ chosen in each subinterval is arbitrary however there are certain obvious choices:

* A *left* Riemann sum is when each $x_i^* = x_{i-1}$ is the left endpoint of the subinterval $[x_{i-1},x_i]$
* A *right* Riemann sum is when each $x_i^* = x_i$ is the right endpoint of the subinterval $[x_{i-1},x_i]$
* A *midpoint* Riemann sum is when each $x_i^* = (x_{i-1} + x_i)/2$ is the midpoint of the subinterval $[x_{i-1},x_i]$

Let's visualize rectangles in the left, right and midpoint Riemann sums for the function

$$
f(x) = \frac{1}{1 + x^2}
$$

over the interval $[0,5]$ with a partition of size $N=10$.

In [ ]:
# let's plot this
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

%config InlineBackend.figure_format = 'retina'
plt.style.use('../lectures/chem1000.mplstyle')

In [ ]:
# our function to integrate
def f(x):
    return 1/(1+x**2)


a = 0; b = 5; N = 10 # range and partion size
n = 10 # Use n*N+1 points to plot the function smoothly

x = np.linspace(a,b,N+1)
y = f(x)

X = np.linspace(a,b,n*N+1)
Y = f(X)

# set up three plots
plt.figure(figsize=(15,5))

# left endpoints
plt.subplot(1,3,1)
plt.plot(X,Y,'b')
x_left = x[:-1] # Left endpoints
y_left = y[:-1]
plt.plot(x_left,y_left,'b.',markersize=10)
plt.bar(x_left,y_left,width=(b-a)/N,alpha=0.2,align='edge',edgecolor='b')
plt.title('Left Riemann Sum, N = {}'.format(N))
plt.ylabel('f(x)')
plt.xlabel('x')

# midpoints
plt.subplot(1,3,2)
plt.plot(X,Y,'b')
x_mid = (x[:-1] + x[1:])/2 # Midpoints
y_mid = f(x_mid)
plt.plot(x_mid,y_mid,'b.',markersize=10)
plt.bar(x_mid,y_mid,width=(b-a)/N,alpha=0.2,edgecolor='b')
plt.title('Midpoint Riemann Sum, N = {}'.format(N))

# right endpoints
plt.subplot(1,3,3)
plt.plot(X,Y,'b')
x_right = x[1:] # right endpoints
y_right = y[1:]
plt.plot(x_right,y_right,'b.',markersize=10)
plt.bar(x_right,y_right,width=-(b-a)/N,alpha=0.2,align='edge',edgecolor='b')
plt.title('Right Riemann Sum, N = {}'.format(N))
plt.ylabel('f(x)')
plt.xlabel('x')

Notice that when the function $f(x)$ is decreasing on $[a,b]$ the left endpoints give an overestimate of the integral $\int_a^b f(x) dx$ and right endpoints give an underestimate. The opposite is true is when the function is increasing.

Let's compute the value of each of the Riemann sums:

In [ ]:
dx = (b-a)/N

print("Partition with",N,"subintervals.")

x_left = np.linspace(a,b-dx,N)
left_riemann_sum = np.sum(f(x_left) * dx)
print("Left Riemann Sum:", left_riemann_sum)

x_midpoint = np.linspace(dx/2,b - dx/2,N)
midpoint_riemann_sum = np.sum(f(x_midpoint) * dx)
print("Midpoint Riemann Sum:", midpoint_riemann_sum)

x_right = np.linspace(dx,b,N)
right_riemann_sum = np.sum(f(x_right) * dx)
print("Right Riemann Sum:", right_riemann_sum)

We know the exact value

$$
\int_0^5 \frac{1}{1 + x^2} dx = \arctan(5)
$$

and we can compare the Riemann sums to the value:

In [ ]:
I = np.arctan(5)
print(I)

In [ ]:
print("Left Riemann Sum Error:",np.abs(left_riemann_sum - I))
print("Midpoint Riemann Sum:",np.abs(midpoint_riemann_sum - I))
print("Right Riemann Sum:",np.abs(right_riemann_sum - I))

Much like the central difference formula for numeric derivatives, the midpoint Riemann Sum is much, much more accurate.

In [ ]:
def riemann_sum(f,a,b,N=50):
    '''Compute the midpoint Riemann sum of f(x) over the interval [a,b].
    f : function - numpy function of one variable
    a , b : numbers - Endpoints of the interval [a,b]
    N : integer - Number of subintervals of equal length in the partition of [a,b]
    '''
    dx = (b - a)/N
    x = np.linspace(a,b,N+1)
    # now we need to get the midpoints
    # this is a bit sneaky:
    # x[:-1] - it takes the array up to the second-to-last element (i.e., last - 1)
    # x[1:] - it takes the second array up to the last
    # so x[0] + x[1], etc.
    x_mid = (x[:-1] + x[1:])/2
    
    # just add up the rectangles
    return np.sum(f(x_mid)*dx)

In [ ]:
riemann_sum(np.sin, 0, np.pi/2, 100)

### Exercise 1

We know that: 

$$
\int_{0}^{1} \frac{4}{1+x^{2}} d x=\pi
$$

Use a Riemann sum to calculate $\pi$.

- How big does N need to be to calculate $\pi$ to 5 decimal places?
- How many intervals do you need to get 10 decimal places correct?


In [ ]:
def f(x):
    return 4/(1 + x**2)

# 3.14159265358979
# 3.14159
# 3.1415926536 (rounding up)
riemann_sum(f, 0, 1, 28000)

In [ ]:
# how big is N for 5 decimal places?
N = 107 # 3.1415999322458608
# how big is n for 10 decimal places?
N = 28000 # gives 3.141592653696086 -- still not accurate enough

### Trapezoid Rule

The [trapezoid rule](https://en.wikipedia.org/wiki/Trapezoidal_rule) gives a better approximation of a definite integral by summing the areas of the trapezoids connecting the points

$$
(x_{i-1},0), (x_i,0), (x_{i-1},f(x_{i-1})), (x_i,f(x_i))
$$

for each subinterval $[x_{i-1},x_i]$ of a partition. Note that the area of each trapezoid is the sum of a rectangle and a triangle

$$
(x_i - x_{i-1}) f(x_{i-1}) + \frac{1}{2}(x_i - x_{i-1}) (f(x_i) - f(x_{i-1})) = \frac{1}{2}(f(x_i) + f(x_{i-1}))(x_i - x_{i-1})
$$

For example, we can use a single trapezoid to approximate:

$$
\int_0^1 e^{-x^2} \, dx
$$

First, let's plot the curve $y = e^{-x^2}$ and the trapezoid on the interval $[0,1]$:

In [ ]:
x = np.linspace(-0.5,1.5,100)
y = np.exp(-x**2)
plt.plot(x,y)

x0 = 0; x1 = 1;
y0 = np.exp(-x0**2); y1 = np.exp(-x1**2);
plt.fill_between([x0,x1],[y0,y1])

plt.xlim([-0.5,1.5]); plt.ylim([0,1.5]);
plt.show()

In [ ]:
A = 0.5*(y1 + y0)*(x1 - x0)
print("Trapezoid area:", A)

Let's compare this to the midpoint Riemann sum (i.e., with one rectangle)

In [ ]:
def f(x):
    return np.exp(-x**2)

riemann_sum(f, 0, 1, 1)

The limit is about:

In [ ]:
riemann_sum(f, 0, 1, 1000)

So one trapezoid sounds like a good idea, but it's around the same accuracy as the midpoint Riemann sum.

Let's try some side-by-side comparisons with an implementation:

In [ ]:
def trapz(f,a,b,N=50):
    '''Approximate the integral of f(x) from a to b by the trapezoid rule.

    The trapezoid rule approximates the integral \int_a^b f(x) dx by the sum:
    (dx/2) \sum_{k=1}^N (f(x_k) + f(x_{k-1}))
    where x_k = a + k*dx and dx = (b - a)/N.

    Parameters
    ----------
    f : function - numpy function of a single variable
    a , b : numbers - Interval of integration [a,b]
    N : integer - Number of subintervals of [a,b]

    '''
    x = np.linspace(a,b,N+1) # N+1 points make N subintervals
    y = f(x)
    y_right = y[1:] # right endpoints
    y_left = y[:-1] # left endpoints
    dx = (b - a)/N
    return (dx/2) * np.sum(y_right + y_left)

In [ ]:
print('Riemann: ', riemann_sum(f, 0, 1, 10))
print('trapezoids: ', trapz(f, 0, 1, 10))

In [ ]:
print('Riemann: ', riemann_sum(np.sin, 0, np.pi/2, 10))
print('trapezoids: ', trapz(np.sin, 0, np.pi/2, 10))

In short, they're really close in accuracy. There's even a numpy (np) method for performing integration:

`np.trapz(y-values, dx=spacing)`

The trick is to make sure that the spacing is consistent between your function and what you tell:

`np.trapz(.., dx=..)`

Note that since `np.trapz()` just asks for $y$ values, it can be used on anything (e.g., real-world data)

In [ ]:
spacing = 0.05
x = np.arange(0, np.pi/2, spacing)
np.trapz(data, dx=spacing)

### Exercise 2

We used Riemann sums to approximate $\pi$ above, let's try with trapezoids

- How many intervals do you need to get 5 decimal place accuracy?
- How many intervals do you need to get 10 decimal place accuracy?

In [ ]:
def f(x):
    return 4/(1 + x**2)

trapz(f, 0, 1, 43083)
# want 3.1415926535 8979

In [ ]:
# how big is N for 5 decimal places?
N = 251 # gives 3.141590008129139
# how big is n for 10 decimal places?
N = 43083 # gives 3.141592653500001

### Simpson's Rule

[Simpson's rule](https://en.wikipedia.org/wiki/Simpson%27s_rule) uses a quadratic polynomial on each subinterval of a partition to approximate the function $f(x)$ and to compute the definite integral. This is an improvement over the trapezoid rule which approximates $f(x)$ by a straight line on each subinterval of a partition.

Image from Wikipedia - function f(x) is in blue and the quadratic approximation P() is in red:
<a href="https://commons.wikimedia.org/wiki/File:Simpsons_method_illustration.svg#/media/File:Simpsons_method_illustration.svg"><img src="https://upload.wikimedia.org/wikipedia/commons/thumb/c/ca/Simpsons_method_illustration.svg/1200px-Simpsons_method_illustration.svg.png" alt="Simpsons method illustration.svg" width="300"></a>

In short, you approximate the integral by taking a small parabola using the two end points and the midpoint (i.e., three points exactly determines a curve).

The formula for Simpson's rule is:

$$
S_N(f) = \frac{\Delta x}{3} \sum_{i=1}^{N/2} \left( f(x_{2i-2}) + 4 f(x_{2i-1}) + f(x_{2i}) \right)
$$

where $N$ is an ***even*** number of subintervals of $[a,b]$, $\Delta x = (b - a)/N$ and $x_i = a + i \Delta x$.

In [ ]:
def simps(f,a,b,N=50):
    '''Approximate the integral of f(x) from a to b by Simpson's rule.

    Simpson's rule approximates the integral \int_a^b f(x) dx by the sum:
    (dx/3) \sum_{k=1}^{N/2} (f(x_{2i-2} + 4f(x_{2i-1}) + f(x_{2i}))
    where x_i = a + i*dx and dx = (b - a)/N.

    Parameters
    ----------
    f : function - numpy function of a single variable
    a , b : numbers - Interval of integration [a,b]
    N : (even) integer - Number of subintervals of [a,b]
    '''
    if N % 2 == 1:
        raise ValueError("N must be an even integer.")
    dx = (b-a)/N
    x = np.linspace(a,b,N+1)
    y = f(x)
    return dx/3 * np.sum(y[0:-1:2] + 4*y[1::2] + y[2::2])

Let's try it out on $\sin x$ first:

In [ ]:
print('Riemann: ', riemann_sum(np.sin, 0, np.pi/2, 4))
print('trapezoids: ', trapz(np.sin, 0, np.pi/2, 4))
print('Simpson: ', simps(np.sin, 0, np.pi/2, 4))

Notice that I intentionally used a small number of intervals here (4) and Simpson's rule is already accurate to over 3 decimal places, much better than either other method.

In some sense, Simpson's rule is a weighted average of the estimates from the two other methods.

There are other numeric integration methods, but Riemann / Trapezoid and Simpson's rule are good, useful methods.

In [ ]:
# from the homework
def f(x):
    return np.sin(x**2)

print('Riemann: ', riemann_sum(f, 0, np.pi/2, 50))
print('trapezoids: ', trapz(f, 0, np.pi/2, 50))
print('Simpson: ', simps(f, 0, np.pi/2, 50))

### Exercise 3

We approximated $\pi$ using Riemann sums and trapezoids above.

Now try with Simpson's rule. 

- How many intervals do you need to get 5 decimal place accuracy?
- How many intervals do you need to get 10 decimal place accuracy?

In [ ]:
def f(x):
    return 4/(1 + x**2)

simps(f, 0, 1, 30)
# want 3.1415926536 or 3.14159265358979

In [ ]:
# how big is N for 5 decimal places?
N = 6 # gives 3.141591780936043
# how big is n for 10 decimal places?
N = 28 # gives 3.1415926535074465
N = 30 # gives 3.141592653535359

### Exercise 4

Go back and compare:

- For Riemann sums, trapezoids, and Simpson's rule, how many intervals did you need for 5 decimal place accuracy?
- For all three methods, how many do you need for 10 decimal place accuracy?

In [ ]:
# 5 decimal places - how big does N need to be?
riemann = 107
trapz = 251
simps = 6
# 10 decimal places - how big does N need to be?
riemann = 28000
trapz = 43083
simps = 28

-------
This notebook is from Prof. Geoffrey Hutchison, University of Pittsburgh
https://github.com/ghutchis/chem1000

Most of this material has been adapted from [Mathematical Python by Patrick Wills](https://github.com/patrickwalls/mathematical-python/tree/master/scipy)

<a rel="license" href="http://creativecommons.org/licenses/by/4.0/"><img alt="Creative Commons License" style="border-width:0" src="https://i.creativecommons.org/l/by/4.0/88x31.png" /></a>